Pororo 라이브러리 설치


In [ ]:
!pip --version

pip 21.1.3 from /usr/local/lib/python3.7/dist-packages/pip (python 3.7)


In [ ]:
!pip install --upgrade pip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 5.1 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3


In [ ]:
!git clone https://github.com/kakaobrain/pororo.git

Cloning into 'pororo'...
remote: Enumerating objects: 2119, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 2119 (delta 11), reused 22 (delta 8), pack-reused 2090
Receiving objects: 100% (2119/2119), 13.09 MiB | 17.74 MiB/s, done.
Resolving deltas: 100% (1448/1448), done.


In [ ]:
ls

pororo/  sample_data/


In [ ]:
cd pororo

/content/pororo


In [ ]:
!pip install -e .
# 이 셀 끝나고 '런타임 다시 시작'

In [ ]:
# 깃허브에서 데이터 다운
!git clone https://github.com/hanmoonje/Korean_ABSA_potential.git

Pororo 사용하여 유의어 변환 후 증강

In [ ]:
import numpy as np
import re
import pandas as pd
import json
from collections import defaultdict
from pororo import Pororo
from tqdm import tqdm
import copy

In [ ]:
data_train = [json.loads(line) for line in open('./Korean_ABSA_potential/data/nikluge-sa-2022-test.jsonl', 'r')] # 원본 데이터
data_aug_back = pd.read_excel("./backtran.xlsx", engine = "openpyxl") # 이전의 역번역 증강데이터

In [ ]:
# 중복 키값을 위한 class 설정
class key_name(object):
  def __init__(self, name):
    self.name = name
  
  def __str__(self):
    return self.name

  def __repr__(self):
    return "'"+self.name+"'"


In [ ]:
# data_train 형식에 따라 data_trian[0] 또는 data_train 사용하기에 여기서 바꿔줌
data_train = [data_train]

In [ ]:
# 제이슨을 딕셔너리로 바꾸기

dic_train = defaultdict(int)

for i in range(len(data_train[0])):
  for j in range(len(data_train[0][i]['annotation'])):#각 순번의 annotation 개수가 다르기에 2번째 for문 사용
    dic_train[key_name(data_train[0][i]['sentence_form'])] = data_train[0][i]['annotation'][j][0], data_train[0][i]['annotation'][j][1], data_train[0][i]['annotation'][j][2]
    # key_name을 사용해 중복 key를 사용함 >> 한문장에 하나의 annotation 만 들어갈 수 있도록

In [ ]:
# 딕셔너리를 DataFrame으로 바꾸는 과정
df = pd.DataFrame(dic_train)
# 행렬 변환
df = df.T
# 인덱스 재설정
df = df.reset_index()
# 열이름 재설정
df = df.rename(columns = {'index':'sentence', 0:'category', 1:'topic',2:'sense'})
df_train = df

In [ ]:
# Pororo 유의어 변환 task 변수 설정
pg = Pororo(task="pg", lang="ko")

In [ ]:
# 조건에 맞는 행의 인덱스 추출

pg_list = []
for i in range(len(df_train['sentence'])):
  if df_train['category'][i] == '브랜드#품질':
    pg_list.append(i)
  elif df_train['category'][i] == '브랜드#디자인':
    pg_list.append(i)
  elif df_train['category'][i] == '브랜드#인지도':
    pg_list.append(i)
  elif df_train['category'][i] == '브랜드#가격':
    pg_list.append(i)
  elif df_train['category'][i] == '제품 전체#다양성':
    pg_list.append(i)
  elif df_train['category'][i] == '패키지/구성품#다양성':
    pg_list.append(i)
  elif df_train['category'][i] == '패키지/구성품#가격':
    pg_list.append(i)
  elif df_train['category'][i] == '패키지/구성품#일반':
    pg_list.append(i)
  elif df_train['category'][i] == '패키지/구성품#품질':
    pg_list.append(i)
  elif df_train['category'][i] == '본품#인지도':
    pg_list.append(i)
  elif df_train['category'][i] == '본품#다양성':
    pg_list.append(i)

In [ ]:
# 조건에 맞는 행 추출
raw = df_train.loc[pg_list]
# 독립된 리스트 만들기
raw = copy.deepcopy(raw)
# 인덱스가 0부터 시작되도록 재설정
raw.reset_index(inplace = True)

In [ ]:
# 유의어 증강 함수 설정하기
def pgpg(faq_question):

  question_based_q = pg(faq_question)

  return question_based_q


In [ ]:
# 원본데이터와 역번역으로 증강한데이터를 붙이기
data_aug_similar = pd.concat([raw,data_aug_back],ignore_index=True)

In [ ]:
# 원본데이터와 이전에 역번역한 데이터를 유의어로 증강하기
for i in tqdm(range(len(data_aug_similar['sentence']))):
  data_aug_similar['sentence'][i] = pgpg(str(data_aug_similar['sentence'][i]))